In [ ]:
from ray.rllib.algorithms.alpha_zero import AlphaZeroConfig
from ray.rllib.algorithms.alpha_zero.models.custom_torch_models import DenseModel
#from ray.rllib.examples.env.cartpole_sparse_rewards import CartPoleSparseRewards
from ray.rllib.models.catalog import ModelCatalog
from ray import rllib, tune
ModelCatalog.register_custom_model("dense_model", DenseModel)
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
instance_path='resources/jsp_instances/standard/ft06.txt'
curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
instance_list=['/resources/jsp_instances/standard/la01.txt','/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
#instance_list=['/resources/jsp_instances/standard/la01.txt']


instance_list=[curr_dir + s for s in instance_list]

def env_creator(config):
    #env = jssp_light_obs_wrapper(jss_lite(instance_path=instance_path))
    env=jssp_light_obs_wrapper_multi_instances(instances_list=instance_list)
    return env

ModelCatalog.register_custom_model("dense_model", DenseModel)    


# use tune to register the custom environment for the ppo trainer
tune.register_env('custom_jssp',env_creator)

# use tune to register the custom environment for the ppo trainer
#tune.register_env('Cart',env_creator)

config = AlphaZeroConfig().training(sgd_minibatch_size=256)\
            .environment(disable_env_checking=True)\
            .resources(num_gpus=0)\
            .environment(env='custom_jssp')\
            .training(model={"custom_model": DenseModel})\
            .rollouts(num_rollout_workers=7)
print(config.to_dict())
# Build a Algorithm object from the config and run 1 training iteration.


trainer = config.build()
print("start training")
#trainer.train()

In [ ]:
import time
for _ in range(6):
    t=time.time()
    trainer.train()
    print(f"finished training:{_} in {time.time()-t}")
    trainer.save_checkpoint('/Users/felix/sciebo/masterarbeit/progra/model-based_rl/training_checkpoints/new_api/multi')

In [ ]:
import ray
ray.shutdown()
from ray.rllib.algorithms.alpha_zero.models.custom_torch_models import DenseModel
tune.register_env('custom_jssp',env_creator)
            #.exploration(explore=False)\
config = AlphaZeroConfig().training(sgd_minibatch_size=256)\
            .resources(num_gpus=0)\
            .exploration(explore=False)\
            .environment(env='custom_jssp')\
            .training(model={"custom_model": DenseModel})\
            .rollouts(num_rollout_workers=4)\
            .training(mcts_config = {
            "puct_coefficient": 1.0,
            "num_simulations": 2,
            "temperature": 1.5,
            "dirichlet_epsilon": 0.00,
            "dirichlet_noise": 0.00,
            "argmax_tree_policy": True,
            "add_dirichlet_noise": False,
        }
        )            
# Build a Algorithm object from the config and run 1 training iteration.
trainer2 = config.build()
trainer2.load_checkpoint('training_checkpoints/new_api/multi/checkpoint-6')
#print(trainer.config)

In [ ]:
env_static = env_creator("s")

obs_static = env_static.reset()
print(env_static.env.instance)


In [ ]:
# test trainer:
#trainer2.load_checkpoint('training_checkpoints/new_api/ft06/checkpoint-6')
for i in range(1,6):
    print(f"checkpoint {i} loaded")
    trainer2.load_checkpoint('training_checkpoints/new_api/multi/checkpoint-'+str(i))
    from copy import deepcopy
    from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
    from ray.rllib.policy.policy_map import PolicyMap
    from ray.rllib.evaluation.episode import MultiAgentEpisode
    import time

    for inst in instance_list:
        env_static=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])    
        length_list=[]
        reward_list=[]
        for _ in range(1):
            policy = trainer2.get_policy(DEFAULT_POLICY_ID)
            action_list=[]

            # env2 is copy for later going evaluation
            env=deepcopy(env_static)
            obs=obs_static

            episode = MultiAgentEpisode(
                PolicyMap(0,0),
                lambda _, __: DEFAULT_POLICY_ID,
                lambda: None,
                lambda _: None,
                0,
            )

            episode.user_data['initial_state'] = env.get_state()

            done = False
            t=time.time()
            while not done:
                action, _, _ = policy.compute_single_action(obs, episode=episode)
                action_list.append(action)
                obs, reward, done, _ = env.step(action)

                episode.length += 1
            print(f"{inst} solved in time: {time.time()-t}s")
            print(reward)

In [ ]:
for inst in instance_list:
        env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
        obs=env.reset()
        print(obs['action_mask'][:env.max_jobs])
        
        obs,reward,a,b = env.step(4)
        print(obs['action_mask'][:env.max_jobs])
        print('cropped')
        print(obs['action_mask'][env.max_jobs:])
    

In [ ]:
from ray.rllib.algorithms.alpha_zero import AlphaZeroConfig
from ray import tune
config = AlphaZeroConfig()
# Print out some default values.
print(config.shuffle_sequences)
# Update the config object.
config.training(lr=tune.grid_search([0.001, 0.0001]))
# Set the config object's env.
config.environment(env="Cart")
# Use to_dict() to get the old-style python config dict
# when running with tune.
tune.run(
    "AlphaZero",
    stop={"episode_reward_mean": 200},
    config=config.to_dict())
    


In [ ]:
print(config)

In [113]:
from copy import deepcopy
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper
from wrapper.jssplight_wrapper import jssp_light_obs_wrapper_multi_instances
from src.jss_lite.jss_lite import jss_lite
import numpy as np
import time
class tree_node:
    def __init__(self, action=None,parent=None):
        self.action=action
        self.children=[]
        self.state=None
        self.parent=parent
        self.observation=None
        self.visited=False
        # extra attributes for search:
        self.leaf_closed=False
class tree_construct:
    def __init__(self):

        curr_dir='/Users/felix/sciebo/masterarbeit/progra/model-based_rl'
        inst=curr_dir + '/resources/jsp_instances/standard/la01.txt'
        instance_list=['/resources/jsp_instances/standard/la01.txt']#,'/resources/jsp_instances/standard/la02.txt','/resources/jsp_instances/standard/la03.txt','/resources/jsp_instances/standard/la04.txt']
        self.env=jssp_light_obs_wrapper_multi_instances(instances_list=[inst])
        # create root
        self.root=tree_node()
        # get initial state into root; todo: change for multi instances, create all childs for instances
        self.root.observation=self.env.reset()
        self.root.state=self.env.get_state()
        self.done=False
        self.reward=0

        # path management
        # open leafs should store the leafs with highest rating
        self.open_leafs=[]
        self.open_leafs.append(self.root)
        self.current_path=self.root
        self.sol_pathes=None
    def insert(self,action=None,parent=None):
        parent.children.append(tree_node(action=action,parent=self))
        self.env.set_state(parent.state)
        parent.children[-1].observation,reward,done,info=self.env.step(action)
        parent.children[-1].state=self.env.get_state()
        if done:
            self.__on_done(reward)

    def __on_done(self,reward):
        self.done=True
        if reward > self.reward:
            self.reward=reward

    def solve(self,max_pathes=None,max_time=None,threshold=0.4):
        # here comes the solving method with abort conditions
        start_time=time.time()
        p_time=time.time()
        self.sol_pathes=0

        if max_pathes==None:
            max_pathes=1000 
        if max_time==None:
            max_time=60
        #print(self.open_leafs)
        while self.open_leafs:
            self.sol_pathes+=1
            # abort conditions:
            node=self.open_leafs.pop()
            if self.sol_pathes > max_pathes or time.time()-start_time > max_time:
                return False
            while not self.done:
                loop=0
                n_r=np.random.uniform(0,1)
                if  n_r < 0.05:
                    #print(3)
                    p_r=3
                elif n_r < 0.1:
                    #print(2)
                    p_r=2
                else:
                    p_r=1
                possible_actions=np.random.choice(len(node.observation['action_mask']), p_r, p=(node.observation['action_mask'] / node.observation['action_mask'].sum()))
                
                for action in set(possible_actions):
                    self.insert(action=action,parent=node)
                for child in node.children:
                    if child != node.children[-1]:
                        #todo: check here if in action mask
                        self.open_leafs.append(child)
                while True:
                    if node.children[-1].visited==False:
                        node.children[-1].visited=True
                        node=node.children[-1]
                        
                        break
                    loop+=1
                    if loop > len(node.children):
                        print("all childrens visited; return done equal true")
                        self.done=True
            self.done=False
            




In [123]:
init=tree_construct()
init.solve()
print(init.reward)
print(init.sol_pathes)

0.9654654654654655
61


In [69]:
import numpy as np
init=tree_construct()
node=init.root
while not init.done:
    
    if np.random.uniform(0,1) < 0.05:
        p_r=2
    else:
        p_r=1
    possible_actions=np.random.choice(len(node.observation['action_mask']), p_r, p=(node.observation['action_mask'] / node.observation['action_mask'].sum()))
    action=possible_actions[0]
        for _ in range(1,len(possible_actions)):
            if possible_actions[_]!=action:
                self.
    init.insert(action=action,parent=node)
    node=node.children[-1]
print(init.reward)

[9 6]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [46]:
print(init.root.observation)
init.env.render()

{'obs': array([1.        , 0.        , 0.21430001, 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.21430001, 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.398     ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.78570002, 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.84689999, 0.        , 0.        , 0.        ,
       1.        , 0.        , 0.551     , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.70410001, 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.38780001,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.1735    , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.78570002, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        ,

In [54]:
np.random.choice(4,2)

array([0, 3])

In [85]:
x=[1]
while x:
    y=x.pop()
    print(y)

1


In [93]:
x=[]
print(x[-1])

IndexError: list index out of range

In [125]:
b=False
def get_r(b):
    if b:
        return True
    else:
        return(print("hello"))
    return False

get_r(b)


hello


In [ ]:
#\.rollouts(num_workers=4)

{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': False, 'num_workers': 2, 'num_envs_per_worker': 1, 'sample_collector': <class 'ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector'>, 'sample_async': False, 'enable_connectors': False, 'rollout_fragment_length': 200, 'batch_mode': 'complete_episodes', 'remote_worker_envs': False, 'remote_env_batch_wait_ms': 0, 'validate_workers_after_construction': True, 'ignore_worker_failures': False, 'recreate_failed_workers': False, 'restart_failed_sub_environments': False, 'num_consecutive_worker_failures_tolerance': 100, 'horizon': None, 'soft_horizon': False, 'no_done_at_end': False, 'preprocessor_pref': 'deepmind', 'observation_filter': 'NoFilter', 'synchronize_filters': True, 'compress_observations': False, 'enable_tf1_exec_eagerly': False, 'sampler_perf_stats_ema_coef': None, 'gamma': 0.99, 'lr': 5e-05, 'train_batch_size': 4000, 'model': {'_use_default_native_models': False, '_disable_preprocessor_api': False, '_disable_action_flattening': False, 'fcnet_hiddens': [256, 256], 'fcnet_activation': 'tanh', 'conv_filters': None, 'conv_activation': 'relu', 'post_fcnet_hiddens': [], 'post_fcnet_activation': 'relu', 'free_log_std': False, 'no_final_linear': False, 'vf_share_layers': True, 'use_lstm': False, 'max_seq_len': 20, 'lstm_cell_size': 256, 'lstm_use_prev_action': False, 'lstm_use_prev_reward': False, '_time_major': False, 'use_attention': False, 'attention_num_transformer_units': 1, 'attention_dim': 64, 'attention_num_heads': 1, 'attention_head_dim': 32, 'attention_memory_inference': 50, 'attention_memory_training': 50, 'attention_position_wise_mlp_dim': 32, 'attention_init_gru_gate_bias': 2.0, 'attention_use_n_prev_actions': 0, 'attention_use_n_prev_rewards': 0, 'framestack': True, 'dim': 84, 'grayscale': False, 'zero_mean': True, 'custom_model': None, 'custom_model_config': {}, 'custom_action_dist': None, 'custom_preprocessor': None, 'lstm_use_prev_action_reward': -1}, 'optimizer': {}, 'explore': True, 'exploration_config': {'type': 'StochasticSampling'}, 'input_config': {}, 'actions_in_input_normalized': False, 'postprocess_inputs': False, 'shuffle_buffer_size': 0, 'output': None, 'output_config': {}, 'output_compress_columns': ['obs', 'new_obs'], 'output_max_file_size': 67108864, 'evaluation_interval': None, 'evaluation_duration': 10, 'evaluation_duration_unit': 'episodes', 'evaluation_sample_timeout_s': 180.0, 'evaluation_parallel_to_training': False, 'evaluation_config': {'mcts_config': {'argmax_tree_policy': True, 'add_dirichlet_noise': False}}, 'off_policy_estimation_methods': {}, 'evaluation_num_workers': 0, 'always_attach_evaluation_results': False, 'enable_async_evaluation': False, 'in_evaluation': False, 'sync_filters_on_rollout_workers_timeout_s': 60.0, 'keep_per_episode_custom_metrics': False, 'metrics_episode_collection_timeout_s': 60.0, 'metrics_num_episodes_for_smoothing': 100, 'min_time_s_per_iteration': None, 'min_train_timesteps_per_iteration': 0, 'min_sample_timesteps_per_iteration': 0, 'logger_creator': None, 'logger_config': None, 'log_level': 'WARN', 'log_sys_usage': True, 'fake_sampler': False, 'seed': None, '_tf_policy_handles_more_than_one_loss': False, '_disable_preprocessor_api': False, '_disable_action_flattening': False, '_disable_execution_plan_api': True, 'simple_optimizer': -1, 'monitor': -1, 'evaluation_num_episodes': -1, 'metrics_smoothing_episodes': -1, 'timesteps_per_iteration': -1, 'min_iter_time_s': -1, 'collect_metrics_timeout': -1, 'buffer_size': -1, 'prioritized_replay': -1, 'learning_starts': 1000, 'replay_batch_size': -1, 'replay_sequence_length': None, 'prioritized_replay_alpha': -1, 'prioritized_replay_beta': -1, 'prioritized_replay_eps': -1, 'min_time_s_per_reporting': -1, 'min_train_timesteps_per_reporting': -1, 'min_sample_timesteps_per_reporting': -1, 'input_evaluation': -1, 'sgd_minibatch_size': 256, 'shuffle_sequences': True, 'num_sgd_iter': 30, 'replay_buffer_config': {'type': 'ReplayBuffer', 'capacity': 1000, 'learning_starts': 500, 'storage_unit': 'fragments'}, 'lr_schedule': None, 'vf_share_layers': False, 'mcts_config': {'puct_coefficient': 1.0, 'num_simulations': 30, 'temperature': 1.5, 'dirichlet_epsilon': 0.25, 'dirichlet_noise': 0.03, 'argmax_tree_policy': False, 'add_dirichlet_noise': True}, 'ranked_rewards': {'enable': True, 'percentile': 75, 'buffer_max_length': 1000, 'initialize_buffer': True, 'num_init_rewards': 100}, 'input': 'sampler', 'multiagent': {'policies': {}, 'policy_map_capacity': 100, 'policy_map_cache': None, 'policy_mapping_fn': None, 'policies_to_train': None, 'observation_fn': None, 'replay_mode': 'independent', 'count_steps_by': 'env_steps'}, 'callbacks': <class 'ray.rllib.algorithms.alpha_zero.alpha_zero.AlphaZeroDefaultCallbacks'>, 'create_env_on_driver': False, 'custom_eval_function': None, 'framework': 'torch', 'num_cpus_for_driver': 1}